In [14]:
from thredds_crawler.crawl import Crawl
from pathlib import Path, PureWindowsPath, PurePath
import geopandas as gpd
import kml2geojson
import geopy

In [26]:
fileserver = 'http://dapds00.nci.org.au/thredds/fileServer/'
catalog = 'http://dapds00.nci.org.au/thredds/catalog/if87/catalog.xml'
# Below for testing - comment out for expected execution
catalog = 'http://dapds00.nci.org.au/thredds/catalog/if87/2015-07-12/catalog.xml'

In [22]:
# Use geopy to get latitude and longitude for intersecting with the MGRS tile polygons
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="useragent")
location = geolocator.geocode("Canberra")
print(location.longitude, location.latitude)
westlon = location.longitude - 0.01
southlat = location.latitude - 0.01
eastlon = location.longitude + 0.01
northlat = location.latitude + 0.01
bbox = (westlon, southlat, eastlon, northlat)

149.1012676 -35.2975906


In [7]:
# Or - define bounding box
westlon = 147
southlat = -38
eastlon = 154
northlat = -27
bbox = (westlon, southlat, eastlon, northlat)
# TODO - implement time selection
# start_date = '2019-12-15'
# end_date = '2020-01-15'


In [ ]:
# Download the MGRS reference tile polygons from ESA
wget.download('https://sentinel.esa.int/documents/247904/1955685/S2A_OPER_GIP_TILPAR_MPC__20151209T095117_V20150622T000000_21000101T000000_B00.kml')

In [5]:
# Geopandas doesn't support direct read of KML so convert to GeoJSON
kml2geojson.convert('S2A_OPER_GIP_TILPAR_MPC__20151209T095117_V20150622T000000_21000101T000000_B00.kml', '.')

In [8]:
# Load MGRS to geopandas using the bbox
mgrs_gdf = gpd.read_file('S2A_OPER_GIP_TILPAR_MPC__20151209T095117_V20150622T000000_21000101T000000_B00.geojson', bbox=bbox)

In [23]:
# mgrs_gdf only contains the intersecting geometry so dump a list of mgrs tile references
mgrs_tile_list = list(mgrs_gdf.name)
# for testing - comment out below for actual run
mgrs_tile_list = ['52JCM']

In [ ]:
# Use THREDDS crawler to get a file listing based on mgrs_tile_list
filelist = []
for i in mgrs_tile_list:
    c = Crawl(catalog, select=[".*"+i+"*/*"])
    for dataset in c.datasets:
        filelist.append([fileserver+dataset.id, './'+dataset.id])

In [ ]:
# Start downloading
filelist
for remotefile in filelist:
    print(remotefile[0])
    Path(remotefile[1]).parent.mkdir(parents=True, exist_ok=True)
    localfile = str(PureWindowsPath(Path(remotefile[1]).absolute()))
    wget.download(remotefile[0], out=localfile)

In [ ]:
#Path('./if87/2015-07-12/S2A_OPER_MSI_ARD_TL_EPA__20160810T192637_A000271_T52JCM_N02.04/NBAR/S2A_OPER_MSI_ARD_TL_EPA__20160810T192637_A000271_T52JCM_NBAR_B01.TIF').parent.mkdir(parents=True, exist_ok=True)
#wget.download('http://dapds00.nci.org.au/thredds/fileServer/if87/2015-07-12/S2A_OPER_MSI_ARD_TL_EPA__20160810T192637_A000271_T52JCM_N02.04/NBAR/S2A_OPER_MSI_ARD_TL_EPA__20160810T192637_A000271_T52JCM_NBAR_B01.TIF', out=str(PureWindowsPath(Path('./if87/2015-07-12/S2A_OPER_MSI_ARD_TL_EPA__20160810T192637_A000271_T52JCM_N02.04/NBAR/S2A_OPER_MSI_ARD_TL_EPA__20160810T192637_A000271_T52JCM_NBAR_B01.TIF').parent)))